# Struct


In [1]:
import sys
sys.path.append("..\..\..") 
from ccg import *

In [2]:
example_struct_def = CStructDef("TExamplestruct", members=[
        CStructDef.Member(CVariable("i8Title", Cint8)),
        CStructDef.Member(CVariable("i8Asdf", Cint8), bitfield=3),
        CStructDef.Member(CArray("i8Name", c_type=Cint8, length=3)),
        CStructDef.Member(
            CVariable("tNestedstruct", c_type=CStructDef(
                # This struct def is anonymous
                is_packed=True,  # And packed
                members=[
                    CStructDef.Member(CVariable("i64Qwer", Cint64)),
                ]),
                      )
        ),

        CStructDef.Member(
            CVariable("tNestedstruct2", c_type=CStructDef(
                "TNestedstruct2",
                members=[
                    CStructDef.Member(CVariable("i64Qwer", Cint64)),
                ]).struct,
                      # Reference the struct type, not the def, and its not declared inplace.
                      # Assumes this struct will be defined somewhere else
                      )
        )
    ])

In [3]:
# Declaration of struct definition
print(example_struct_def.declaration())



struct TExamplestruct
{
	int8_t i8Title;
	int8_t i8Asdf: 3;
	int8_t i8Name[3];
	struct __attribute__((__packed__))
	{
		int8_t i64Qwer;
	} tNestedstruct;
	struct TNestedstruct2 tNestedstruct2;
}


In [4]:
# Can declare a struct with a variable in the same sentence
print(CVariable("tInst", c_type=example_struct_def).declaration())

struct TExamplestruct
{
	int8_t i8Title;
	int8_t i8Asdf: 3;
	int8_t i8Name[3];
	struct __attribute__((__packed__))
	{
		int8_t i64Qwer;
	} tNestedstruct;
	struct TNestedstruct2 tNestedstruct2;
} tInst;


In [5]:
# Or assume the struct is already declared and use it as type
print(CVariable("tInst", c_type=example_struct_def.struct).declaration())

struct TExamplestruct tInst;


In [6]:
# Can do a typedef of the struct with the declaration of the struct inplace
print(example_struct_def.type("TMyStruct").typedef())

typedef struct TExamplestruct
{
	int8_t i8Title;
	int8_t i8Asdf: 3;
	int8_t i8Name[3];
	struct __attribute__((__packed__))
	{
		int8_t i64Qwer;
	} tNestedstruct;
	struct TNestedstruct2 tNestedstruct2;
} TMyStruct;


In [7]:
# Or the struct is already declared and can be typedefed afterwards
print(example_struct_def.struct.type("TMyStruct").typedef())

typedef struct TExamplestruct TMyStruct;


In [8]:
# Third member is the packed anonymous struct, can be reobtained
packed_struct = example_struct_def.members[3].variable.c_type
if not isinstance(packed_struct, CStructDef):
    raise TypeError

In [9]:
# Only packed structs have bit_sizes, calculated. CCG Cant know the size of structs with possible padding
print(packed_struct.bit_size)

64
